<a href="https://colab.research.google.com/github/tomato-sugar/python_lesson/blob/main/magenta_basic/magenta_basic1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2022年 東北TECH道場 郡山道場 第7期

##Magentaを用いた機械学習作曲入門（１）

##《このセクションで学ぶこと》
- Magentaとは何かについて簡単に説明します。
- Melody RNN で単音のシンプルなメロディーの自動作曲を試みます。

##《Magentaの概要》

###(1) Magentaとは
Google AI によって運営されている、「音楽とアートのための機械学習の研究プロジェクト及び音楽機械学習ライブラリ」のことで、作曲だけではなく絵画作成やドローイング認識などのさまざまな芸術分野の開発が行われています。主な特徴は以下のようになります。なお、[公式サイト](https://magenta.tensorflow.org/)には[Google Colab によるデモ](https://magenta.tensorflow.org/demos/colab/)も公開されています。
- Magentaを使用すると、機械学習などの知識がなくても自動作曲が可能である。
- TensorFlow（Googleの機械学習プラットフォーム）をベースに開発されている。
- 学習済みモデルが搭載されており、自ら学習させることなく自動作曲を行うことが可能である。
- 学習モデルには、音楽家であるバッハのような曲を生成するPolyphony RNN やピアノロールを使用し現代音楽風の曲を生成する Pianoroll RNN など様々なモデルがある。


###(2) Magentaに用意されているAI作曲モデル
Magentaに用意されているAI作曲モデルには、主に以下のようなものがあります。詳しくは[こちら](https://github.com/magenta/magenta/tree/main/magenta/models)を参照してください。  
- Melody RNN：単音のシンプルなメロディーの作曲
- Drums RNN：ドラム演奏パートの作曲
- Music VAE：3パートのバンド演奏作曲
- Improv RNN：コード進行に沿ったアドリブメロディーの作曲
- Polyphony RNN：バッハ風合唱曲の作曲
- Pianoroll RNN：現代音楽のような複雑な和音の作曲
- Peformance RNN：高度なピアノ演奏の作曲  など



##《自動作曲の実行手順》
まずは、簡単な曲を作ってみましょう。以下の処理を順番に進めてください。

###(1) ライブラリのインストール
次のライブラリをインストールします。少し時間がかかります。
- Magenta：機械学習による音楽生成ライブラリ
- pyFluidSynth：音楽を生成するためのソフトウェア シンセサイザー
- pretty_midi：MIDI データを処理するためのユーティリティ

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

###(2) 基本となる音を与える
Magentaに作曲してもらうには、基本となる音を1音以上引数で渡す必要があります。また、NoteSequenceという専用のデータ形式に変換する必要があります。ここでは、「ド・レ・ミ」の3音をNoteSequenceのnotesに追加してから、きちんと追加されたかを再生して確認しています。

In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

# 基本の音を格納するbase_musicを作成
base_music = music_pb2.NoteSequence()

# base_musicに音を追加
# pitch：音の周波数  start_time：音の開始時間  end_time：終了時間  velocity：音の速さ
base_music.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)  # 「ド」
base_music.notes.add(pitch=62, start_time=0.5, end_time=1.0, velocity=80)  # 「レ」
base_music.notes.add(pitch=64, start_time=1.0, end_time=1.5, velocity=80)  # 「ミ」

base_music.total_time = 1.5  # 所要時間
base_music.tempos.add(qpm=120.0)  # 曲のテンポ（4分音符が1分間に120）

# 再生
note_seq.play_sequence(base_music, synth=note_seq.fluidsynth)  

###(3) Melody RNN で単音のシンプルなメロディーの作曲
RNNとは、リカレントニューラルネットワーク（recurrent neural network）のことで、日本語では「再帰型ニューラルネットワーク」と呼ばれます。これは、ディープラーニングなどの機械学習で、多層のニューラルネットワークの一部に再帰的な手続きを導入したものです。こちら（[RNNの概要を理解しよう](https://atmarkit.itmedia.co.jp/ait/articles/1804/25/news143.html)）がわかりやすいと思うので、ご参照ください。つまり、Melody RNN では、直近の音符の並びから次の音符を予測することを繰り返して曲を生成しています。ですから、基本となる音が最低1音は必要になります。<br>なお、Melody RNN の詳細については、[こちら](https://github.com/magenta/magenta/tree/main/magenta/models/melody_rnn)を確認してください。

####● 学習済みモデルの準備
Magentaには、学習済みモデルが用意されていて、それを利用することで機械学習を行うこと無く作曲が可能です。Melody RNN には、以下の4つのモデルが用意されています。

- basic_rnn.mag：基本的なメロディーの生成
- mono_rnn.mag：パターンに沿ったメロディーの生成
- lookback_rnn.mag：時間的整合性のあるメロディーの生成
- attention_rnn.mag：MIDIの全音域をカバーするメロディーの生成

ここでは、「basic_rnn」を「/magenta/models/」フォルダにダウンロードして保存してから利用します。保存したモデルファイルは、Google Colabのサイドバーの「ファイル」で存在を確認することができます。**もし、表示されていない場合には、ファイルの「更新」をしてください。**

In [ ]:
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# 学習済みモデルの初期化
note_seq.notebook_utils.download_bundle(  # 学習済みデータのダウンロード
    "basic_rnn.mag", "magenta/models/"
)
bundle = sequence_generator_bundle.read_bundle_file(  # 学習済みデータの読み込み
    "magenta/models/basic_rnn.mag"
)
generator_map = melody_rnn_sequence_generator.get_generator_map()  # 生成器のmapの作成
melody_rnn = generator_map["basic_rnn"](checkpoint=None, bundle=bundle)  # mapから生成器の設定
melody_rnn.initialize()  # 初期化

####● 自動作曲
上記で生成した生成器を使って作曲を行います。ここで指定するのは、ベースになる音と、生成したい曲の秒数、曲のランダム度合いです。実行するたびに生成される曲は異なります。

In [ ]:
from note_seq.protobuf import generator_pb2

base_seq = base_music  # ベースになる音
total_sec = 30  # 生成したい曲の秒数（自由に変更可能）
temperature = 1.2  # 曲のランダム度合い（これを大きくするとランダム度合いが大きくなる）
base_end_time = base_music.total_time  #ベース曲の終了時刻

# 生成器のオプションの設定
options = generator_pb2.GeneratorOptions()  # 生成器のオプション
options.args["temperature"].float_value = temperature  # ランダム度合い
options.generate_sections.add(
    start_time = base_end_time,  # 作曲開始時刻
    end_time = total_sec  # 作曲終了時刻
)

# 曲の生成
gen_seq = melody_rnn.generate(base_seq, options)

# 再生
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)

####● 生成した音楽データを保存
作曲した曲を再利用するために、MIDIデータに変換してから保存します。保存したファイルは、「/content/magenta/」に保存されるので、左サイドメニューの「ファイル」からアクセスしてダウンロードしてください。**もし、ファイルが表示されていない場合には、ファイルの「更新」をしてください。**<br>再生アプリには、[musescore3](https://musescore.org/ja) がお勧めです。無料で利用でき、再生だけではなく音符の確認や変更も可能です。

In [5]:
# MIDIデータに変換し保存
note_seq.sequence_proto_to_midi_file(
    gen_seq, "magenta/melody_rnn1.mid"
)  

##《演習》
1. 学習済みモデルを変更する<br>「学習済みモデルの準備」の項を変更して、それぞれダウンロードして自動作曲してみましょう。
2. 基本となる音を入力する<br>基本となる音をキーボードから入力できるようにしましょう。次のような関数を作って「(2) 基本となる音を与える」で呼び出して利用してみましょう。




In [ ]:
from IPython.core.display import set_matplotlib_formats


def base_music_create(base_music):
    start = 0.0
    total_time = 0.0
    # 音階とピッチの辞書
    scale_to_pich = {
        "ド": 60,
        "レ": 62,
        "ミ": 64,
        "ファ": 65,
        "ソ": 67,
        "ラ": 69,
        "シ": 71,
        "ド^": 72,
    }

    print("カタカナで「ド」～「ド^」を「,」区切りで入力してください。")
    print("【例】ソ,ミ,ミ,ファ,ミ,レ,ド,ソ,ミ,ミ,レ")
    onpu = input("音符＞ ")
    onpu = onpu.split(",")
    for on in onpu:
        if on in scale_to_pich:
            base_music.notes.add(
                pitch=scale_to_pich[on],
                start_time=start,
                end_time=start + 0.5,
                velocity=80
            )
            start += 0.5
            total_time += 0.5

    # 所要時間の返却
    return total_time


##《参考文献》
- 斎藤 喜寛 「[Magentaで開発 AI作曲](https://www.ohmsha.co.jp/book/9784274227318/)」 オーム社  2021/07/20出版
- 吾妻 幸長  「[【Magenta+Colab】AIによる作曲を学ぼう！ -ディープラーニングで自動生成する音楽データ](https://www.udemy.com/course/ai-music/)」Udemy